In [6]:
# we want to generate a report for each cave. 

from pylatex import Document, Section, Subsection, Command, Figure
from pylatex.utils import italic, NoEscape


In [7]:
# a basic document

doc = Document("basic")

In [8]:
with doc.create(Section("Overview")):
    doc.append("Cave name: ")
    doc.append(italic("La Cueva Meillor"))

In [9]:
with doc.create(Subsection("A figure")):
    with doc.create(Figure(position="h")) as figure:
        figure.add_image("3rdGallery_stats.pdf", width = "450px")
        figure.add_caption("Look, a figure.")
        

In [10]:
tex = doc.dumps()

In [11]:
tex

'\\documentclass{article}%\n\\usepackage[T1]{fontenc}%\n\\usepackage[utf8]{inputenc}%\n\\usepackage{lmodern}%\n\\usepackage{textcomp}%\n\\usepackage{lastpage}%\n\\usepackage{graphicx}%\n%\n%\n%\n\\begin{document}%\n\\normalsize%\n\\section{Overview}%\n\\label{sec:Overview}%\nCave name: %\n\\textit{La Cueva Meillor}\n\n%\n\\subsection{A figure}%\n\\label{subsec:Afigure}%\n\n\n\\begin{figure}[h]%\n\\centering%\n\\includegraphics[width=450px]{3rdGallery_stats.pdf}%\n\\caption{Look, a figure.}%\n\\end{figure}\n\n%\n\\end{document}'

In [ ]:
doc.dump